In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import seaborn as sns

from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error

pd.set_option("display.precision", 3)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 15)

In [36]:
def well_display_logs(dataset, wellname, fmname, net_var1, net_var2, ref_depth, depth_step, print_flag, path):
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    depth = data['TST']
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT']
    net_init = data[net_var1]*100 
    net = data[net_var2]*100
    perm = data['LPERM']
    perm1 = data['PERM_DS']
    perm2 = data['PERM_US']
    calcite = data['calc']*100

    net_init_sum = (net_init.sum()*0.001).round(1)
    net_sum = (net.sum()*0.001).round(1)

    fig, ax = plt.subplots(1,4, figsize=(7,11), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[0].set_title(net_var1)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.33, zorder=5)
    twin0.vlines(0.5, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    twin0.xaxis.set_ticks(np.arange(0, 1.5, 0.5))
    twin0.set_xlim(0, 1.5)
    twin0.grid(True)
    twin0.plot(net_init, depth, color='orange', linewidth=0.5, alpha=1)
    twin0.fill_betweenx(depth,net_init, color='orange', alpha=0.25)
    ax[1].plot(rhob, depth, color='red', alpha=0.75) 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].grid(True)
    ax[1].set_xlim(1.65, 2.65)
    ax[1].plot(calcite, depth, color='#4ebef1', linewidth=0.5, alpha=1)
    ax[1].fill_betweenx(depth,calcite, color='#4ebef1', alpha=0.5)
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue', alpha=0.75)
    twin1.set_xlim(0.6, 0)
    twin1.grid(False)
    ax[2].plot(phit, depth, color='green', linestyle='dashed')
    ax[2].set_xlim(0.3, 0)
    ax[2].xaxis.set_ticks(np.arange(0, 0.3, 0.1))
    ax[2].tick_params(labelsize=10)
    ax[2].grid(True)
    ax[2].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    ax[2].set_title(net_var2)
    twin2 = ax[2].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5, alpha=1)
    twin2.fill_betweenx(depth,net, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    twin2.xaxis.set_ticks([])
    twin2.grid(False)
    twin3 = ax[2].twiny()
    twin3.plot(rdeep, depth, color='black', linewidth=1, alpha=1)
    twin3.set_xscale('log')
    twin3.set_xlim(1, 200)
    twin3.grid(False)
    ax[3].plot(perm, depth, color='purple', alpha=0.75)
    ax[3].plot(perm1, depth, color='purple', alpha=0.5)
    ax[3].plot(perm2, depth, color='purple', alpha=0.5)
    ax[3].set_xscale('log')
    ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + 
                 ' net_vsh: ' + str(net_init_sum) + 'm' 
                 ' net: ' + str(net_sum) + 'm', 
                 fontsize=14)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + wellname + '.png', dpi=300)
    else:
        pass
    plt.show()


# Bal8

In [ ]:
def calcite_data_preparation8(cutoff_list):
    df_bal8_azr_vshclp2 = pd.read_csv('C:\jupyter\SPP\inputoutput\df_bal8_azr_v2.csv').drop('Unnamed: 0', axis=1)

    df_bal8_azr_vshclp2 = df_bal8_azr_vshclp2[~df_bal8_azr_vshclp2.well.isin(['E20Y', 'E10Z', 'J13X'])]
    well_phit_flag8 = df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.phit_flag==1].groupby('well')['phit_flag'].apply(lambda x: x.iloc[0]).reset_index().well.unique()

    # npss_cutoff = 0.25
    # rhob_cutoff = 2.45
    df_bal8_azr_vshclp2['calc'] = 0.0
    df_bal8_azr_vshclp2.loc[((df_bal8_azr_vshclp2.RHOB >= cutoff_list[1]) & 
                            (df_bal8_azr_vshclp2.VSH <=0.5) &
                            (df_bal8_azr_vshclp2.NPSS > 0) &
                            (df_bal8_azr_vshclp2.NPSS < cutoff_list[0]) &
                            (df_bal8_azr_vshclp2.NET_VSH == 1)), 'calc'] = 1
    
    def calcite_remover8(dataset):
        dataset.loc[((dataset.well == 'C33Z') &
                    (dataset.TST > 145) &
                    (dataset.TST < 147)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C33Z') &
                    (dataset.TST > 170) &
                    (dataset.TST < 172)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C31') &
                    (dataset.TST > 1980) &
                    (dataset.TST < 1992)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C33') &
                    (dataset.TST > 1598) &
                    (dataset.TST < 1600)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C33') &
                    (dataset.TST > 1632) &
                    (dataset.TST < 1634)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C14Z') &
                    (dataset.TST > 60) &
                    (dataset.TST < 70)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C14Z') &
                    (dataset.TST > 206) &
                    (dataset.TST < 208)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C42') &
                    (dataset.TST > 2059) &
                    (dataset.TST < 2093)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C42') &
                    (dataset.TST > 2105) &
                    (dataset.TST < 2110)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C42') &
                    (dataset.TST > 2115) &
                    (dataset.TST < 2120)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C42') &
                    (dataset.TST > 2183) &
                    (dataset.TST < 2207)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C28') &
                    (dataset.TST > 1754) &
                    (dataset.TST < 1764)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C28') &
                    (dataset.TST > 1804) &
                    (dataset.TST < 1808)), 'calc'] = 0
        dataset.loc[((dataset.well == 'D02Y') &
                    (dataset.TST > 340) &
                    (dataset.TST < 443)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C18') &
                    (dataset.RHOB >= 2.39) & 
                    (dataset.VSH <=0.5) &
                    (dataset.NPSS > 0) &
                    (dataset.NPSS < cutoff_list[0]) &
                    (dataset.NET_VSH == 1)), 'calc'] = 1
        dataset.loc[((dataset.well == 'C18') &
                    (dataset.TST > 1561) &
                    (dataset.TST < 1563)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C18') &
                    (dataset.TST > 1601) &
                    (dataset.TST < 1603)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C18') &
                    (dataset.TST > 1609) &
                    (dataset.TST < 1617)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C11') &
                    (dataset.TST > 2067) &
                    (dataset.TST < 2070)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C11') &
                    (dataset.TST > 2140) &
                    (dataset.TST < 2144)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C01AY') &
                    (dataset.TST > 428) &
                    (dataset.TST < 430)), 'calc'] = 0
        dataset.loc[((dataset.well == 'D36') &
                    (dataset.TST > 2860) &
                    (dataset.TST < 2870)), 'calc'] = 0
        dataset.loc[((dataset.well == 'D27') &
                    (dataset.TST > 2198) &
                    (dataset.TST < 2200)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C29Y') &
                    (dataset.TST > 422) &
                    (dataset.TST < 425)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C03Z') &
                    (dataset.RHOB >= 2.4) & 
                    (dataset.VSH <=0.5) &
                    (dataset.NPSS > 0) &
                    (dataset.NPSS < cutoff_list[0]) &
                    (dataset.NET_VSH == 1)), 'calc'] = 1
        dataset.loc[((dataset.well == 'C02') &
                    (dataset.TST > 1863) &
                    (dataset.TST < 1932.5)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C02') &
                    (dataset.TST > 1934) &
                    (dataset.TST < 1949.5)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C09') &
                    (dataset.RHOB >= 2.35) & 
                    (dataset.VSH <=0.5) &
                    (dataset.NPSS > 0) &
                    (dataset.NPSS < cutoff_list[0]) &
                    (dataset.NET_VSH == 1)), 'calc'] = 1
        dataset.loc[((dataset.well == 'D04') &
                    (dataset.TST > 2921) &
                    (dataset.TST < 3027)), 'calc'] = 0
        dataset.loc[((dataset.well == 'D33') &
                    (dataset.TST > 2909) &
                    (dataset.TST < 2917)), 'calc'] = 0
        dataset.loc[((dataset.well == 'D33') &
                    (dataset.TST > 2990) &
                    (dataset.TST < 2996)), 'calc'] = 0
        dataset.loc[((dataset.well == 'D29') &
                    (dataset.TST > 2460) &
                    (dataset.TST < 2470)), 'calc'] = 0   
        dataset.loc[((dataset.well == 'D29') &
                    (dataset.TST > 2481) &
                    (dataset.TST < 2509)), 'calc'] = 0
        dataset.loc[((dataset.well == 'D09Y') &
                    (dataset.TST > 790) &
                    (dataset.TST < 917)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C40') &
                    (dataset.TST > 1869) &
                    (dataset.TST < 1987)), 'calc'] = 0
        dataset.loc[((dataset.well == 'D03Z') &
                    (dataset.TST > 313) &
                    (dataset.TST < 380)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C03ST1') &
                    (dataset.TST > 712) &
                    (dataset.TST < 714)), 'calc'] = 0
        dataset.loc[((dataset.well == 'D37') &
                    (dataset.TST > 2844) &
                    (dataset.TST < 2846)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C39') &
                    (dataset.TST > 2189) &
                    (dataset.TST < 2191)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C34') &
                    (dataset.TST > 1965) &
                    (dataset.TST < 1973)), 'calc'] = 0
        dataset.loc[((dataset.well == 'C34') &
                    (dataset.TST > 2023) &
                    (dataset.TST < 2037)), 'calc'] = 0                             
        return dataset
    df_bal8_azr_vshclp2 = calcite_remover8(df_bal8_azr_vshclp2)

    df_bal8_azr_vshclp2['calc_net'] = 0
    df_bal8_azr_vshclp2.loc[(df_bal8_azr_vshclp2.calc == 1) & (df_bal8_azr_vshclp2.NET == 1), 'calc_net'] = 1
    df_bal8_azr_vshclp2['calc_net'] = df_bal8_azr_vshclp2['calc_net'].astype(float)
   
    return df_bal8_azr_vshclp2

cutoff_list = [0.25, 2.45]
df_bal8_azr_v2 = calcite_data_preparation8(cutoff_list)
df_bal8_azr_v2.to_csv('C:\jupyter\SPP\inputoutput\df_bal8_azr_v2_calc.csv')

calc_cutoff_8 = 0.5
def calcite_map8_calculation(df_bal8_azr_vshclp2, calc_cutoff):
    calcite_map8 = df_bal8_azr_vshclp2.groupby('well')[['Xmean', 
                                                        'Ymean',
                                                        'calc', 
                                                        'calc_net', 
                                                        'field']].agg(
                                                                    {'calc_net':'sum',
                                                                     'calc':'sum', 
                                                                     'Xmean':lambda x: x.iloc[0], 
                                                                     'Ymean':lambda x: x.iloc[0],
                                                                     'field':lambda x: x.iloc[0]}).reset_index().rename(columns={
                                                                         'calc':'calc_sum',
                                                                         'calc_net':'calc_net_sum',})
    wells_map8 = df_bal8_azr_vshclp2.groupby('well')[['Xmean','Ymean','TVD_SCS']].apply(lambda x: x.iloc[0]).reset_index()
    wells_map8 = wells_map8[~wells_map8.well.isin(['E20Y', 'E10Z', 'J13X'])]
    calcite_map8['calc_sum'] = calcite_map8['calc_sum']*0.1
    calcite_map8['calc_net_sum'] = calcite_map8['calc_net_sum']*0.1                                                          
    calcite_map8 = calcite_map8[calcite_map8.calc_sum > calc_cutoff]
    return calcite_map8, wells_map8 
calcite_map8, wells_map8 = calcite_map8_calculation(df_bal8_azr_v2, calc_cutoff_8)

def calcite_display8(calcite_map, wells_map, width_sel=1200, height_sel=600):
    calcite_points = go.Scatter(    x=calcite_map['Xmean'], 
                                    y=calcite_map['Ymean'], 
                                    mode='markers', 
                                    marker=dict(
                                                color = '#4ebef1',
                                                opacity=0.5,
                                                size=calcite_map['calc_sum'], 
                                                sizemode='area', 
                                                sizeref=2.*max(calcite_map['calc_sum'])/(40.**2), 
                                                sizemin=4,
                                                line=dict(width=1, color='black')),
                                    customdata = calcite_map[['well', 'calc_sum']].round(1),
                                    hovertemplate="".join(["well:%{customdata[0]}, calc %{customdata[1]}<extra></extra>"])
                                )
    calcite_net_points = go.Scatter(    x=calcite_map['Xmean'], 
                                        y=calcite_map['Ymean'], 
                                        mode='markers', 
                                        marker=dict(
                                                    color = '#0a69d6',
                                                    opacity=0.5,
                                                    size=calcite_map['calc_net_sum'], 
                                                    sizemode='area', 
                                                    sizeref=2.*max(calcite_map['calc_sum'])/(40.**2), 
                                                    sizemin=4
                                                    ),
                                        customdata = calcite_map[['well', 'calc_net_sum']].round(1),
                                        hovertemplate="".join(["well:%{customdata[0]}, calc_net %{customdata[1]}<extra></extra>"])
                                )
    well_points = go.Scatter(   x=wells_map['Xmean'], 
                                y=wells_map['Ymean'], 
                                mode='markers',
                                marker=dict(symbol='cross',
                                            color='black',
                                            size=5
                                            ),
                                customdata = wells_map[['well','TVD_SCS']],
                                hovertemplate="".join(["well:%{customdata[0]},tvd:%{customdata[1]}<extra></extra>"])
                                ) 
    fig = go.Figure()  
    fig.add_trace(calcite_points)
    fig.add_trace(calcite_net_points)
    fig.add_trace(well_points)
    fig.update_layout(  
                        # autosize=True,
                        autosize=False,
                        width=width_sel,
                        height=height_sel,
                        margin=dict(l=10, r=10, b=10, t=50),
                        showlegend=False,
                        title = 'Calcite distribution map Bal8 with calcite_sum cutoff 0.5m',
                        title_font = dict(size=20)
                        )
    fig.show()
    # fig.write_html(r'C:\jupyter\SPP\plots\calcite_map8.html')

calcite_display8(calcite_map8, wells_map8)

In [43]:
# for wellname in df_bal8_azr_v2.well.unique():
#     well_display_logs(df_bal8_azr_v2, wellname, 'Balakhany VIII', 'NET_VSH', 'NET', 'TST', 5, 'print', 'C:\\jupyter\\SPP\\plots\\logs_vsh_bal8\\')

In [6]:
def well_display_logs_calc8(dataset, wellname, fmname, net_var, cutoff_list, comments, ref_depth, depth_step, print_flag, path):
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    depth = data[ref_depth]
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT']
    net = data['NET']*10
    net_vsh = data['NET_VSH']*10 
    calc = data[net_var]*10
    perm = data['LPERM']
    perm1 = data['PERM_DS']
    perm2 = data['PERM_US']

    calcite_sum = data['calc'].sum()
    calcite_sum = (calcite_sum*0.1).round(1)

    fig, ax = plt.subplots(1,4, figsize=(7,10), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[0].grid(True)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.33, zorder=5)
    twin0.vlines(0.5, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    twin0.xaxis.set_ticks(np.arange(0, 1.5, 0.5))
    twin0.set_xlim(0, 1.5)
    twin0.grid(True)
    twin0.plot(net_vsh, depth, color='orange', linewidth=0.1)
    twin0.fill_betweenx(depth,net_vsh, color='orange', alpha=0.25)
    ax[1].plot(rhob, depth, color='red', alpha=0.75)
    ax[1].vlines(cutoff_list[1], ymin=min(depth), ymax=max(depth), color='#bb2f2f', linestyle='dashed', linewidth=1) 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].grid(True)
    ax[1].set_xlim(1.65, 2.65)
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue', alpha=0.75)
    twin1.vlines(cutoff_list[0], ymin=min(depth), ymax=max(depth), color='blue', linestyle='dashed', linewidth=1)
    twin1.set_xlim(0.6, 0)
    twin1.grid(False)
    twin1.plot(net, depth, color='orange', linewidth=0.1)
    twin1.fill_betweenx(depth,net, color='orange', alpha=0.25)
    ax[2].plot(phit, depth, color='green', linestyle='dashed')
    ax[2].set_xlim(0.3, 0)
    ax[2].xaxis.set_ticks(np.arange(0, 0.3, 0.1))
    ax[2].tick_params(labelsize=10)
    ax[2].grid(True)
    ax[2].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    twin2 = ax[2].twiny()
    twin2.plot(calc, depth, color='#6ccbf3', linewidth=0.1)
    twin2.fill_betweenx(depth,calc, color='#6ccbf3', alpha=0.5)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    twin2.xaxis.set_ticks([])
    twin2.grid(False)
    twin3 = ax[2].twiny()
    twin3.plot(rdeep, depth, color='black', linewidth=1, alpha=1)
    twin3.set_xscale('log')
    twin3.set_xlim(1, 200)
    twin3.grid(False)
    ax[3].plot(perm, depth, color='purple', alpha=0.75)
    ax[3].plot(perm1, depth, color='purple', alpha=0.5)
    ax[3].plot(perm2, depth, color='purple', alpha=0.5)
    ax[3].set_xscale('log')
    ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(calcite_sum), fontsize=14)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + wellname + '.png')
    else:
        pass
    plt.show()

# for wellname in calcite_map8.well.unique():
#     well_display_logs_calc8(df_bal8_azr_vshclp2, wellname, 'Balakhany VIII', 'calc', 
                            # cutoff_list, 'test', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\plots\\calcite_bal8\\')

In [101]:
ntd8 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\layers\ntd_top_phi_bot8_bp.csv').drop('Unnamed: 0', axis=1)
ntd8 = ntd8[(ntd8.well.str.startswith('C') | (ntd8.well.str.startswith('D')))]

def calcite_df_prepar8(ntd8, calcite_map8, calc_cutoff):
    ntd8_calc = ntd8.set_index('well').join(calcite_map8.set_index('well')).reset_index().drop(['Xmean','Ymean'], axis=1).fillna(0)
    ntd8_calc['phit_htst'] = ntd8_calc['phit_avg'] * ntd8_calc['htst']
    ntd8_calc['vsh_htst'] = ntd8_calc['vsh_avg'] * ntd8_calc['htst']
    ntd8_calc['khtst'] = ntd8_calc['perm_avg'] * ntd8_calc['htst']
    ntd8_calc_v2 = ntd8_calc.set_index('well').join(ntd8_calc.groupby('well')['htst'].sum().reset_index().rename(
                                                    columns={'htst':'htst_sum'}).set_index('well')).reset_index()

    ntd8_calc_v3 = ntd8_calc_v2.groupby('well').agg({'perm_avg': 'mean',
                                                     'khtst':'sum', 
                                                     'phit_htst': 'sum', 
                                                     'vsh_htst':'sum',
                                                     'htst_sum': lambda x: x.iloc[0],
                                                     'calc_sum': lambda x: x.iloc[0],
                                                     'calc_net_sum': lambda x: x.iloc[0],}).reset_index()
    ntd8_calc_v3['phit_wavg'] = ntd8_calc_v3['phit_htst'] / ntd8_calc_v3['htst_sum']
    ntd8_calc_v3['vsh_wavg'] = ntd8_calc_v3['vsh_htst'] / ntd8_calc_v3['htst_sum']
    ntd8_calc_v3['calc_flag'] = 0
    ntd8_calc_v3.loc[ntd8_calc_v3.calc_sum > calc_cutoff, 'calc_flag'] = 1

    ntd8_calc_v3['calc_sum_perc'] = ntd8_calc_v3['calc_sum'] / ntd8_calc_v3['htst_sum']
    ntd8_calc_v3['calc_net_sum_perc'] = ntd8_calc_v3['calc_net_sum'] / ntd8_calc_v3['htst_sum']
    field = df_bal8_azr_vshclp2.groupby('well')['field'].apply(lambda x: x.iloc[0]).reset_index()
    ntd8_calc_v3 = ntd8_calc_v3.set_index('well').join(field.set_index('well')).reset_index()

    return ntd8_calc_v3
ntd8_calc_v3 = calcite_df_prepar8(ntd8, calcite_map8, calc_cutoff_8)

ntd8_calc_v3.loc[ntd8_calc_v3.well == 'C27Y', 'field'] = 'WEST AZERI'
ntd8_calc_v3.loc[ntd8_calc_v3.well.isin(['D09Y','D19Z']), 'field'] = 'EAST AZERI'
ntd8_calc_v3.set_index('well').join(df_bal8_azr_v2.groupby('well')[['Xmean','Ymean']].first()).to_csv(
            r'C:\jupyter\SPP\inputoutput\calcite\ntd8_calc_v3.csv')

In [ ]:
def calc8_display():
    fig, ax = plt.subplots(1,3, figsize=(18,5))
    custom_palette = {0:'#088a28', 1:'#05dcf8'}
    sns.kdeplot(ntd8_calc_v3, x='khtst', hue='calc_flag', log_scale=True, ax=ax[0], palette=custom_palette, bw_adjust =0.75)
    ax[0].grid(True)
    sns.kdeplot(ntd8_calc_v3, x='phit_wavg', hue='calc_flag', ax=ax[1], palette=custom_palette, bw_adjust =0.75)
    ax[1].grid(True)
    sns.scatterplot(data=ntd8_calc_v3, x='phit_wavg', y='htst_sum', 
                    hue='calc_flag', ax=ax[2], palette=custom_palette,
                    edgecolor='black', s=50, alpha=0.5)
    ax[2].grid(True, lw=0.5)
    plt.show();
calc8_display()

In [ ]:
def curve_fitting8(ntd8_calc_v3):
    def linear(x, a, b):
        return a * x + b

    def quadratic(x, a, b, c):
        return a * x**2 + b * x + c

    data = ntd8_calc_v3[ntd8_calc_v3.calc_sum > 0]
    x_data = data['calc_sum_perc'] 
    y_data = data['khtst'] 

    # Use curve_fit to fit the functions to the data
    popt_linear, pcov_linear = curve_fit(linear, x_data, y_data)
    popt_quadratic, pcov_quadratic = curve_fit(quadratic, x_data, y_data)

    # Calculate the MSE for each model
    mse_linear = mean_squared_error(y_data, linear(x_data, *popt_linear)).round(0)
    mse_quadratic = mean_squared_error(y_data, quadratic(x_data, *popt_quadratic)).round(0)

    # Use curve_fit to fit the functions to the data
    popt_linear, pcov_linear = curve_fit(linear, x_data, y_data)
    popt_quadratic, pcov_quadratic = curve_fit(quadratic, x_data, y_data)
    print("MSE for linear model: ", mse_linear)
    print("MSE for quadratic model: ", mse_quadratic)
    correlation = x_data.corr(y_data).round(2)
    print('Correlation khtst=f(calc_sum_perc)', correlation)

    fig, ax = plt.subplots(1,2, figsize=(14,5))
    sns.scatterplot(data=ntd8_calc_v3[ntd8_calc_v3.calc_sum > 0], x='calc_sum_perc', y='khtst', 
                    color='#4ebef1', edgecolor='black', s=50, alpha=0.5, ax=ax[0])
    sns.scatterplot(data=ntd8_calc_v3[ntd8_calc_v3.calc_sum > 0], x='calc_net_sum_perc', y='khtst', 
                    color='#0a69d6', edgecolor='black', s=50, alpha=0.5, ax=ax[1])
    sns.lineplot(x= x_data, y=quadratic(x_data, *popt_quadratic), ax=ax[0], color='red', lw=2,
                label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt_quadratic))
    for ind, row in ntd8_calc_v3[ntd8_calc_v3.calc_sum > 0].iterrows():
        ax[0].text(row['calc_sum_perc']+0.005, row['khtst'], row['well'], fontsize=6, ha='left')
        ax[1].text(row['calc_net_sum_perc'], row['khtst'], row['well'], fontsize=6, ha='left')
    ax[0].grid(True)
    ax[0].set_title('Calcite sum_perc vs khtst')
    ax[1].set_title('Calcite sum_net_perc vs khtst')
    ax[1].grid(True)
    plt.show();
curve_fitting8(ntd8_calc_v3)

In [ ]:
def calc8_plotly():
    calc_wells = ntd8_calc_v3[ntd8_calc_v3.calc_flag == 1]
    layers = ntd8_calc_v3[ntd8_calc_v3.calc_flag == 0]

    layers = go.Scatter(x=layers['phit_wavg'], 
                        y=layers['htst_sum'],
                        mode='markers', 
                        marker=dict(color='#088a28', 
                                    size=10,
                                    opacity=0.5,
                                    line=dict(color='black', width=0.5)),
                            text=layers[['well', 'phit_wavg', 'htst_sum']].round(1),
                            hovertemplate="".join(["well:%{text[0]}, phit:%{text[1]}, htst:%{text[2]}<extra></extra>"])
                        )
    calcite = go.Scatter(x=calc_wells['phit_wavg'],
                        y=calc_wells['htst_sum'],
                        mode='markers', 
                        marker=dict(   color='#05dcf8', 
                                        size=calc_wells['calc_sum'], 
                                        sizemode='area', 
                                        sizeref=4*max(calc_wells['calc_sum'])/(40**2), 
                                        sizemin=4,
                                        opacity=0.75,
                                        line=dict(color='black', width=0.5)),
                        text=calc_wells[['well', 'phit_wavg', 'htst_sum', 'calc_sum']].round(2),
                        hovertemplate="".join(["well:%{text[0]}," + 
                                               "phit:%{text[1]}," +
                                               "htst:%{text[2]}," +
                                               "calc:%{text[3]},<extra></extra>"])
                        )
    calcite_net_points = go.Scatter(    x=calc_wells['phit_wavg'], 
                                        y=calc_wells['htst_sum'], 
                                        mode='markers', 
                                        marker=dict(
                                                    color = '#0a69d6',
                                                    opacity=0.5,
                                                    size=calc_wells['calc_net_sum'], 
                                                    sizemode='area', 
                                                    sizeref=4*max(calc_wells['calc_sum'])/(40**2), 
                                                    sizemin=4
                                                    ),
                                        customdata = calc_wells[['well', 'calc_net_sum']].round(1),
                                        hovertemplate="".join(["well:%{customdata[0]}, calc_net %{customdata[1]}<extra></extra>"])
                                )       
    fig = go.Figure()
    fig.add_traces(layers)
    fig.add_traces(calcite)
    fig.add_traces(calcite_net_points)
    fig.update_xaxes(title_text='phit_wavg')
    fig.update_yaxes(title_text='htst_sum')
    fig.update_layout(  autosize=False,
                        width=600,
                        height=600,
                        showlegend=False,
                        margin=dict(l=10, r=10, b=10, t=50),
                        title = 'Phit vs Htst with htst of calcite Balakhany VIII',)
    fig.show()
calc8_plotly()

In [ ]:
def cross_plot_calculation8():
    df_net_calc8 = df_bal8_azr_v2.groupby('well')[['NET_VSH','NET','calc']].agg({'NET_VSH':'sum', 'NET':'sum', 'calc':'sum'}).reset_index()
    df_net_calc8['calc'] = df_net_calc8['calc']*0.1
    df_net_calc8['NET_VSH'] = df_net_calc8['NET_VSH']*0.1
    df_net_calc8['NET'] = df_net_calc8['NET']*0.1
    df_net_calc8 = df_net_calc8[df_net_calc8.calc > 0.5]
    df_net_calc8['diff_netvsh_net'] = df_net_calc8['NET_VSH'] - df_net_calc8['NET']
    return df_net_calc8
df_net_calc8 = cross_plot_calculation8()

def df_net_calc_display8():
    net = go.Scatter(x=df_net_calc8['NET'], 
                    y=df_net_calc8['NET_VSH'], 
                    mode='markers', 
                    marker=dict(color='green', size=10, opacity=0.5,
                                line=dict(width=1, color='black')
                                ),
                    hovertext=df_net_calc8[['well','NET','NET_VSH', 'diff_netvsh_net', 'calc']].round(1),
                    hovertemplate="".join(["well:%{hovertext[0]}, " + 
                                            "net:%{hovertext[1]}, " +
                                            "netvsh:%{hovertext[2]}, " +
                                            "diff:%{hovertext[3]}, " + 
                                            "calc:%{hovertext[4]} " +
                                            "<extra></extra>"])
                    )
    line = go.Scatter(x=[30, 75], y=[30, 75], mode='lines', line=dict(color='black', width=1))
    fig=go.Figure()
    fig.add_trace(net)
    fig.add_trace(line)
    fig.update_layout(title='NET vs NET_VSH', 
                    xaxis_title='NET', 
                    yaxis_title='NET_VSH',
                    width=600,
                    height=600,
                    showlegend=False,
                    margin=dict(l=10, r=10, b=10, t=50))
    fig.show()

df_net_calc_display8()
df_net_calc8.head(3)

# Bal10

In [ ]:
def calcite_data_preparation10(coef_list):
    df_bal10_vshclp2 = pd.read_csv('C:\jupyter\SPP\inputoutput\general_logs\df_bal10_vshclp2.csv').drop('Unnamed: 0', axis=1)
    well_phit_flag10 = df_bal10_vshclp2[df_bal10_vshclp2.phit_flag==1].groupby('well')['phit_flag'].apply(lambda x: x.iloc[0]).reset_index().well.unique()

    def avg_rhob_npss_calc(df_bal10_vshclp2):
        rhob_avg = df_bal10_vshclp2[df_bal10_vshclp2.RHOB > 0 ].groupby(['well','NET'])['RHOB'].mean().reset_index().rename(columns={'RHOB':'RHOB_sh_avg'})
        rhob_avg = rhob_avg[rhob_avg.NET == 0].drop('NET', axis=1).round(2)
        npss_avg = df_bal10_vshclp2[df_bal10_vshclp2.RHOB > 0 ].groupby(['well','NET'])['NPSS'].mean().reset_index().rename(columns={'NPSS':'NPSS_ss_avg'})
        npss_avg = npss_avg[npss_avg.NET == 1].drop('NET', axis=1).round(2)
        df_bal10_vshclp2 = (df_bal10_vshclp2.merge(rhob_avg, on='well', how='left')).merge(npss_avg, on='well', how='left')
        return df_bal10_vshclp2
    df_bal10_vshclp2 = avg_rhob_npss_calc(df_bal10_vshclp2)

    df_bal10_vshclp2['calc'] = 0.0
    df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.RHOB >= df_bal10_vshclp2.RHOB_sh_avg + coef_list[0]) & 
                            (df_bal10_vshclp2.VSH <=0.5) &
                            (df_bal10_vshclp2.NPSS > 0) &
                            (df_bal10_vshclp2.NPSS < df_bal10_vshclp2.NPSS_ss_avg - coef_list[1]) &
                            (df_bal10_vshclp2.NET_VSH == 1)), 'calc'] = 1

    def calcite_remover10():
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'A05') &
                                (df_bal10_vshclp2.TST > 1272) &
                                (df_bal10_vshclp2.TST < 1287)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'A07') &
                                (df_bal10_vshclp2.TST > 663) &
                                (df_bal10_vshclp2.TST < 670)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'A07Y') &
                                (df_bal10_vshclp2.TST > 47) &
                                (df_bal10_vshclp2.TST < 115)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'A07Z') &
                                (df_bal10_vshclp2.TST > 606) &
                                (df_bal10_vshclp2.TST < 616)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'A07Z') &
                                (df_bal10_vshclp2.TST > 621) &
                                (df_bal10_vshclp2.TST < 641)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'A07Z') &
                                (df_bal10_vshclp2.TST > 683) &
                                (df_bal10_vshclp2.TST < 686)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'A08Z') &
                                (df_bal10_vshclp2.TST > 942) &
                                (df_bal10_vshclp2.TST < 962)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'A08Z') &
                                (df_bal10_vshclp2.TST > 972) &
                                (df_bal10_vshclp2.TST < 1017)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'A09') &
                                (df_bal10_vshclp2.TST > 708) &
                                (df_bal10_vshclp2.TST < 723)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'A09') &
                                (df_bal10_vshclp2.TST > 729) &
                                (df_bal10_vshclp2.TST < 735)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'A10') &
                                (df_bal10_vshclp2.TST > 1236) &
                                (df_bal10_vshclp2.TST < 1301)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'A12') &
                                (df_bal10_vshclp2.TST > 662) &
                                (df_bal10_vshclp2.TST < 712)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'A12W') &
                                (df_bal10_vshclp2.TST > 309.8) &
                                (df_bal10_vshclp2.TST < 314.8)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'A12W') &
                                (df_bal10_vshclp2.TST > 307) &
                                (df_bal10_vshclp2.TST < 309.5)), 'calc'] = 1
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'A12W') &
                                (df_bal10_vshclp2.TST > 330.3) &
                                (df_bal10_vshclp2.TST < 331.3)), 'calc'] = 1
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'B08') &
                                (df_bal10_vshclp2.TST > 2119) &
                                (df_bal10_vshclp2.TST < 2249)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'C14Z') &
                                (df_bal10_vshclp2.TST > 407) &
                                (df_bal10_vshclp2.TST < 411)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'C33Z') &
                                (df_bal10_vshclp2.TST > 482) &
                                (df_bal10_vshclp2.TST < 490)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'D01Y') &
                                (df_bal10_vshclp2.TST > 730) &
                                (df_bal10_vshclp2.TST < 755)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'D02Z') &
                                (df_bal10_vshclp2.TST > 511) &
                                (df_bal10_vshclp2.TST < 559)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'D03') &
                                (df_bal10_vshclp2.TST > 3165) &
                                (df_bal10_vshclp2.TST < 3239)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'D23Y') &
                                (df_bal10_vshclp2.TST > 737) &
                                (df_bal10_vshclp2.TST < 745)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'E04') &
                                (df_bal10_vshclp2.TST > 22.5) &
                                (df_bal10_vshclp2.TST < 42.5)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'E16X') &
                                (df_bal10_vshclp2.TST > 682) &
                                (df_bal10_vshclp2.TST < 713)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'E16X') &
                                (df_bal10_vshclp2.TST > 724.8) &
                                (df_bal10_vshclp2.TST < 782)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'E23') &
                                (df_bal10_vshclp2.TST > 2841) &
                                (df_bal10_vshclp2.TST < 2863)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'E25') &
                                (df_bal10_vshclp2.TST > 2231) &
                                (df_bal10_vshclp2.TST < 2237)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'E33') &
                                (df_bal10_vshclp2.TST > 2733) &
                                (df_bal10_vshclp2.TST < 2837)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'E38') &
                                (df_bal10_vshclp2.TST > 520) &
                                (df_bal10_vshclp2.TST < 580)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'F01Z') &
                                (df_bal10_vshclp2.TST > -80) &
                                (df_bal10_vshclp2.TST < -20)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'G06') &
                                (df_bal10_vshclp2.TST > 1938) &
                                (df_bal10_vshclp2.TST < 1994)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'H07') &
                                (df_bal10_vshclp2.TST > 1798) &
                                (df_bal10_vshclp2.TST < 1806)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'H07') &
                                (df_bal10_vshclp2.TST > 1818) &
                                (df_bal10_vshclp2.TST < 1836)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'J08') &
                                (df_bal10_vshclp2.TST > 1956) &
                                (df_bal10_vshclp2.TST < 2054)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'J09') &
                                (df_bal10_vshclp2.TST > 1737) &
                                (df_bal10_vshclp2.TST < 1745)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'J09') &
                                (df_bal10_vshclp2.TST > 1762) &
                                (df_bal10_vshclp2.TST < 1776)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'J09') &
                                (df_bal10_vshclp2.TST > 1778) &
                                (df_bal10_vshclp2.TST < 1789)), 'calc'] = 0
        df_bal10_vshclp2.loc[(  (df_bal10_vshclp2.well == 'J13') &
                                (df_bal10_vshclp2.TST > 2317) &
                                (df_bal10_vshclp2.TST < 2323)), 'calc'] = 1
        return df_bal10_vshclp2
    df_bal10_vshclp2 = calcite_remover10()
    
    df_bal10_vshclp2['calc_net'] = 0
    df_bal10_vshclp2.loc[(df_bal10_vshclp2.calc == 1) & (df_bal10_vshclp2.NET == 1), 'calc_net'] = 1
    df_bal10_vshclp2['calc_net'] = df_bal10_vshclp2['calc_net'].astype(float)
    
        
   
    return df_bal10_vshclp2
coef_list = [0.015, 0.02]
df_bal10_vshclp2 = calcite_data_preparation10(coef_list)
# df_bal10_vshclp2.to_csv(r'C:\jupyter\SPP\inputoutput\general_logs\df_bal10_vshclp2_calc.csv')

calc_cutoff_10 = 0.5
def calcite_map10_calculation(df_bal10_vshclp2, calc_cutoff):
    calcite_map10 = df_bal10_vshclp2.groupby('well')[['Xmean',
                                                      'Ymean',
                                                      'calc',
                                                      'calc_net', 
                                                      'field']].agg(
                                                                    {'calc_net':'sum',
                                                                     'calc':'sum', 
                                                                     'Xmean':lambda x: x.iloc[0], 
                                                                     'Ymean':lambda x: x.iloc[0],
                                                                     'field':lambda x: x.iloc[0]}).reset_index().rename(columns={
                                                                         'calc':'calc_sum',
                                                                         'calc_net':'calc_net_sum'})
    wells_map10 = df_bal10_vshclp2.groupby('well')[['Xmean','Ymean','TVD_SCS']].apply(lambda x: x.iloc[0]).reset_index()
    calcite_map10['calc_sum'] = calcite_map10['calc_sum']*0.1
    calcite_map10['calc_net_sum'] = calcite_map10['calc_net_sum']*0.1                                                          
    calcite_map10 = calcite_map10[calcite_map10.calc_sum > calc_cutoff]
    return calcite_map10, wells_map10
calcite_map10, wells_map10 = calcite_map10_calculation(df_bal10_vshclp2, calc_cutoff_10)

def calcite_display10(calcite_map, wells_map, width_sel=1200, height_sel=600):
    calcite_points = go.Scatter(    x=calcite_map['Xmean'], 
                                    y=calcite_map['Ymean'], 
                                    mode='markers', 
                                    marker=dict(
                                                color = '#4ebef1',
                                                opacity=0.5,
                                                size=calcite_map['calc_sum'], 
                                                sizemode='area', 
                                                sizeref=2.*max(calcite_map['calc_sum'])/(40.**2), 
                                                sizemin=4,
                                                line=dict(width=1, color='black')),
                                    customdata = calcite_map[['well', 'calc_sum']].round(1),
                                    hovertemplate="".join(["well:%{customdata[0]}, calc %{customdata[1]}<extra></extra>"])
                                )
    calcite_net_points = go.Scatter(    x=calcite_map['Xmean'], 
                                        y=calcite_map['Ymean'], 
                                        mode='markers', 
                                        marker=dict(
                                                    color = '#0a69d6',
                                                    opacity=0.5,
                                                    size=calcite_map['calc_net_sum'], 
                                                    sizemode='area', 
                                                    sizeref=2.*max(calcite_map['calc_sum'])/(40.**2), 
                                                    sizemin=4
                                                    ),
                                        customdata = calcite_map[['well', 'calc_net_sum']].round(1),
                                        hovertemplate="".join(["well:%{customdata[0]}, calc_net %{customdata[1]}<extra></extra>"])
                                )
    well_points = go.Scatter(  x=wells_map['Xmean'], 
                            y=wells_map['Ymean'], 
                            mode='markers',
                            marker=dict(symbol='cross',
                                        color='black',
                                        size=5
                                        ),
                            customdata = wells_map[['well','TVD_SCS']],
                            hovertemplate="".join(["well:%{customdata[0]},tvd:%{customdata[1]}<extra></extra>"])
                            ) 
    fig = go.Figure()  
    fig.add_trace(calcite_points)
    fig.add_trace(calcite_net_points)
    fig.add_trace(well_points)
    fig.update_layout(  
                        # autosize=True,
                        autosize=False,
                        width=width_sel,
                        height=height_sel,
                        margin=dict(l=10, r=10, b=10, t=50),
                        showlegend=False,
                        title = 'Calcite distribution map Bal10 with calcite_sum cutoff 0.5m',
                        title_font = dict(size=20)
                        )
    fig.show()
    # fig.write_html('C:\jupyter\SPP\plots\calcite_map10.html')

calcite_display10(calcite_map10, wells_map10, width_sel=1400, height_sel=800)

In [44]:
# for wellname in df_bal10_vshclp2.well.unique():
#     well_display_logs(df_bal10_vshclp2, wellname, 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'print', 'C:\\jupyter\\SPP\\plots\\logs_vsh_bal10\\')

In [ ]:
ntd10 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\layers\ntd_top_phi_bot10_bp.csv').drop('Unnamed: 0', axis=1)

def calcite_df_prepar10(ntd10, calcite_map10, calc_cutoff):
    ntd10_calc = ntd10.set_index('well').join(calcite_map10.set_index('well')).reset_index().drop(['Xmean','Ymean'], axis=1).fillna(0)
    ntd10_calc['phit_htst'] = ntd10_calc['phit_avg'] * ntd10_calc['htst']
    ntd10_calc['vsh_htst'] = ntd10_calc['vsh_avg'] * ntd10_calc['htst']
    ntd10_calc['khtst'] = ntd10_calc['perm_avg'] * ntd10_calc['htst']
    ntd10_calc_v2 = ntd10_calc.set_index('well').join(
                                                        ntd10_calc.groupby('well')['htst'].sum().reset_index().rename(
                                                        columns={'htst':'htst_sum'}).set_index('well')
                                                        ).reset_index()

    ntd10_calc_v3 = ntd10_calc_v2.groupby('well').agg({ 'perm_avg': 'mean',
                                                        'khtst':'sum', 
                                                        'phit_htst': 'sum', 
                                                        'vsh_htst':'sum',
                                                        'htst_sum': lambda x: x.iloc[0],
                                                        'calc_sum': lambda x: x.iloc[0],
                                                        'calc_net_sum': lambda x: x.iloc[0],}).reset_index()
    ntd10_calc_v3['phit_wavg'] = ntd10_calc_v3['phit_htst'] / ntd10_calc_v3['htst_sum']
    ntd10_calc_v3['vsh_wavg'] = ntd10_calc_v3['vsh_htst'] / ntd10_calc_v3['htst_sum']
    ntd10_calc_v3['calc_flag'] = 0
    ntd10_calc_v3.loc[ntd10_calc_v3.calc_sum > calc_cutoff, 'calc_flag'] = 1

    ntd10_calc_v3['calc_sum_perc'] = ntd10_calc_v3['calc_sum'] / ntd10_calc_v3['htst_sum']
    ntd10_calc_v3['calc_net_sum_perc'] = ntd10_calc_v3['calc_net_sum'] / ntd10_calc_v3['htst_sum']
    field = df_bal10_vshclp2.groupby('well')['field'].apply(lambda x: x.iloc[0]).reset_index()
    ntd10_calc_v3 = ntd10_calc_v3.set_index('well').join(field.set_index('well')).reset_index()

    return ntd10_calc_v3

ntd10_calc_v3 = calcite_df_prepar10(ntd10, calcite_map10, calc_cutoff_10)
ntd10_calc_v3.loc[ntd10_calc_v3.well == 'C27Y', 'field'] = 'WEST AZERI'
ntd10_calc_v3.loc[ntd10_calc_v3.well.isin(['D09Y','D19Z']), 'field'] = 'EAST AZERI'
ntd10_calc_v3.loc[ntd10_calc_v3.well == 'E12Z', 'field'] = 'DWG'
ntd10_calc_v3.loc[ntd10_calc_v3.well == 'J13X', 'field'] = 'WEST CHIRAG'
ntd10_calc_v3.set_index('well').join(df_bal10_vshclp2.groupby('well')[['Xmean','Ymean']].first()).to_csv(
                r'C:\jupyter\SPP\inputoutput\calcite\ntd10_calc_v3.csv')

def calc8_display():
    fig, ax = plt.subplots(1,3, figsize=(18,5))
    custom_palette = {0:'#088a28', 1:'#05dcf8'}
    sns.kdeplot(ntd10_calc_v3, x='khtst', hue='calc_flag', log_scale=True, ax=ax[0], palette=custom_palette, bw_adjust =0.75)
    ax[0].grid(True)
    sns.kdeplot(ntd10_calc_v3, x='phit_wavg', hue='calc_flag', ax=ax[1], palette=custom_palette, bw_adjust =0.75)
    ax[1].grid(True)
    sns.scatterplot(data=ntd10_calc_v3, x='phit_wavg', y='htst_sum', 
                    hue='calc_flag', ax=ax[2], palette=custom_palette,
                    edgecolor='black', s=50, alpha=0.5)
    ax[2].grid(True, lw=0.5)
    plt.show();
calc8_display()

In [ ]:
def curve_fitting10(ntd10_calc_v3):
    data = ntd10_calc_v3[ntd10_calc_v3.calc_sum > 0]
    x_data = data['calc_sum_perc'] 
    y_data = data['khtst'] 
    correlation = x_data.corr(y_data).round(2)
    print('Correlation khtst=f(calc_sum_perc)', correlation)

    fig, ax = plt.subplots(1,2, figsize=(14,5))
    sns.scatterplot(data=ntd10_calc_v3[ntd10_calc_v3.calc_sum > 0], x='calc_sum_perc', y='khtst', 
                    color='#4ebef1', edgecolor='black', s=50, alpha=0.5, ax=ax[0])
    sns.scatterplot(data=ntd10_calc_v3[ntd10_calc_v3.calc_net_sum > 0], x='calc_net_sum_perc', y='khtst', 
                    color='#0a69d6', edgecolor='black', s=50, alpha=0.5, ax=ax[1])

    for ind, row in ntd10_calc_v3[ntd10_calc_v3.calc_sum > 0].iterrows():
        ax[0].text(row['calc_sum_perc']+0.005, row['khtst'], row['well'], fontsize=6, ha='left')
    for ind, row in ntd10_calc_v3[ntd10_calc_v3.calc_net_sum > 0].iterrows():
        ax[1].text(row['calc_net_sum_perc'], row['khtst'], row['well'], fontsize=6, ha='left')
    ax[0].grid(True)
    ax[0].set_title('Calcite sum_perc vs khtst')
    ax[1].set_title('Calcite sum_net_perc vs khtst')
    ax[1].grid(True)
    plt.show();

curve_fitting10(ntd10_calc_v3)

In [ ]:
def calc10_plotly():
    calc_wells = ntd10_calc_v3[ntd10_calc_v3.calc_flag == 1]
    layers = ntd10_calc_v3[ntd10_calc_v3.calc_flag == 0]

    layers = go.Scatter(x=layers['phit_wavg'], 
                        y=layers['htst_sum'],
                        mode='markers', 
                        marker=dict(color='#088a28', 
                                    size=10,
                                    opacity=0.5,
                                    line=dict(color='gray', width=0.5)),
                            text=layers[['well', 'phit_wavg', 'htst_sum']].round(1),
                            hovertemplate="".join(["well:%{text[0]}, phit:%{text[1]}, htst:%{text[2]}<extra></extra>"])
                        )
    calcite = go.Scatter(x=calc_wells['phit_wavg'],
                        y=calc_wells['htst_sum'],
                        mode='markers', 
                        marker=dict(   color='#05dcf8', 
                                        size=calc_wells['calc_sum'], 
                                        sizemode='area', 
                                        sizeref=4*max(calc_wells['calc_sum'])/(40**2), 
                                        sizemin=4,
                                        opacity=0.75,
                                        line=dict(color='gray', width=0.5)),
                        text=calc_wells[['well', 'phit_wavg', 'htst_sum', 'calc_sum']].round(2),
                        hovertemplate="".join(["well:%{text[0]}, phit:%{text[1]}, htst:%{text[2]}, calc:%{text[3]}<extra></extra>"])
                        )
    calcite_net_points = go.Scatter(    x=calc_wells['phit_wavg'], 
                                        y=calc_wells['htst_sum'], 
                                        mode='markers', 
                                        marker=dict(
                                                    color = '#0a69d6',
                                                    opacity=0.5,
                                                    size=calc_wells['calc_net_sum'], 
                                                    sizemode='area', 
                                                    sizeref=4*max(calc_wells['calc_sum'])/(40**2), 
                                                    sizemin=4
                                                    ),
                                        customdata = calc_wells[['well', 'calc_net_sum']].round(1),
                                        hovertemplate="".join(["well:%{customdata[0]}, calc_net %{customdata[1]}<extra></extra>"])
                                )          
    fig = go.Figure()
    fig.add_traces(layers)
    fig.add_traces(calcite)
    fig.add_traces(calcite_net_points)
    fig.update_xaxes(title_text='phit_wavg')
    fig.update_yaxes(title_text='htst_sum')
    fig.update_layout(  autosize=False,
                        width=600,
                        height=600,
                        showlegend=False,
                        margin=dict(l=10, r=10, b=10, t=50),
                        title = 'Phit vs Htst with htst of calcite Balakhany X',)
    fig.show()
calc10_plotly()

In [32]:
# well_display_logs(df_bal10_vshclp2, 'A12', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\output\\')

In [ ]:
def well_display_logs_calc10(dataset, wellname, fmname, net_var, coef_list, comments, ref_depth, depth_step, print_flag, path):
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    depth = data[ref_depth]
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT']
    net = data['NET']*10
    net_vsh = data['NET_VSH']*10 
    calc = data[net_var]*10
    perm = data['LPERM']
    perm1 = data['PERM_DS']
    perm2 = data['PERM_US']
    rhob_avg = data['RHOB_sh_avg'] + coef_list[0]
    npss_avg = data['NPSS_ss_avg'] - coef_list[1]

    # rhob_avg = data[data.RHOB > 0 ].groupby('NET')['RHOB'].mean().reset_index()
    # rhob_avg = rhob_avg[rhob_avg.NET==0]['RHOB'].iloc[0].round(2) + 0.025
    # npss_avg = data[data.NPSS > 0].groupby('NET')['NPSS'].mean().reset_index()
    # npss_avg = npss_avg[npss_avg.NET==1]['NPSS'].iloc[0].round(2) - 0.03

    calcite_sum = data['calc'].sum()
    calcite_sum = (calcite_sum*0.1).round(1)

    fig, ax = plt.subplots(1,4, figsize=(7,12), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[0].grid(True)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.33, zorder=5)
    twin0.vlines(0.5, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    twin0.xaxis.set_ticks(np.arange(0, 1.5, 0.5))
    twin0.set_xlim(0, 1.5)
    twin0.grid(True)
    twin0.plot(net_vsh, depth, color='orange', linewidth=0.1)
    twin0.fill_betweenx(depth,net_vsh, color='orange', alpha=0.25)
    ax[1].plot(rhob, depth, color='red', alpha=0.75)
    ax[1].vlines(rhob_avg, ymin=min(depth), ymax=max(depth), color='#bb2f2f', linestyle='dashed', linewidth=0.75) 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].grid(True)
    ax[1].set_xlim(1.65, 2.65)
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue', alpha=0.75)
    twin1.vlines(npss_avg, ymin=min(depth), ymax=max(depth), color='blue', linestyle='dashed', linewidth=0.75)
    twin1.set_xlim(0.6, 0)
    twin1.grid(False)
    twin1.plot(net, depth, color='orange', linewidth=0.1)
    twin1.fill_betweenx(depth,net, color='orange', alpha=0.25)
    ax[2].plot(phit, depth, color='green', linestyle='dashed')
    ax[2].set_xlim(0.3, 0)
    ax[2].xaxis.set_ticks(np.arange(0, 0.3, 0.1))
    ax[2].tick_params(labelsize=10)
    ax[2].grid(True)
    ax[2].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    twin2 = ax[2].twiny()
    twin2.plot(calc, depth, color='#6ccbf3', linewidth=0.1)
    twin2.fill_betweenx(depth,calc, color='#6ccbf3', alpha=0.5)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    twin2.xaxis.set_ticks([])
    twin2.grid(False)
    twin3 = ax[2].twiny()
    twin3.plot(rdeep, depth, color='black', linewidth=1, alpha=1)
    twin3.set_xscale('log')
    twin3.set_xlim(1, 200)
    twin3.grid(False)
    ax[3].plot(perm, depth, color='purple', alpha=0.75)
    ax[3].plot(perm1, depth, color='purple', alpha=0.5)
    ax[3].plot(perm2, depth, color='purple', alpha=0.5)
    ax[3].set_xscale('log')
    ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(calcite_sum), fontsize=14)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + wellname + '.png')
    else:
        pass
    plt.show()

for wellname in calcite_map10.well.unique()[:7]:
    well_display_logs_calc10(df_bal10_vshclp2, wellname, 'Balakhany X', 'calc', coef_list, 'test', 'TST', 5, 'print', 'C:\\jupyter\\SPP\\plots\\calcite_bal10\\')